<a href="https://colab.research.google.com/github/okadatsuneo/GBFS_Import/blob/main/gbfstocsv_hc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#モジュールのインポート
import pandas as pd
import json

In [ ]:
# Install XlsxWriter library
!pip install XlsxWriter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.9/152.9 kB 3.2 MB/s eta 0:00:00


In [ ]:
#ドライブのマウント
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Googleドライブにあるstation_information.jsonを開く
path = '/content/drive/MyDrive/python/'
file_name_station_info = 'h_station_information.json'
file_path = path + file_name_station_info
with open(file_path) as f:
    json_dict = json.load(f)

#'stations'の値を新たなリストlist_stationとする。
list_station = json_dict['data']['stations']

#list_stationの内容をDataFrameにする。
df = pd.DataFrame(list_station)

#ここからはHelloCyclingのファイルに必要な処理
#list_stationの中の'vehicle_type_capacity'の値をリストにする。
l_capacity = [ d['vehicle_type_capacity'] for d in list_station]
#さらにその中の'num_bikes_limit'の値をリストにする。
l_limitcap = [ d['num_bikes_limit'] for d in l_capacity]

#l_limitcapのリストをdfに変換する
df_cap =pd.DataFrame(l_limitcap, columns=['capacity'])

#dfにdf_capを列として追加する。:df2
df2 = df.join(df_cap)

#'system_id'の列を作り、"<system_id>:<station_id>"のように結合する
df2['system_id'] = 'hellocycling:'
df2['ref:gbfs'] = df2['system_id'] + df2['station_id'] 

#インポートしやすいように'address'から都道府県名を取り出し、'addr:province'として列を作成する。
for index, row in df2.iterrows():
  #検索行の'address'の値を変数に入れる:address_check
  address_check = df2.at[index,'address']
  #3番目の文字が"都"の場合
  if address_check[2:3] == "都":
    #3番めまでの文字を'addr:province'に格納する。
    df2.at[index,'addr:province'] = address_check[:3]
  #3番目の文字が"道"の場合
  elif address_check[2:3] == "道":
    #3番めまでの文字を'addr:province'に格納する。
    df2.at[index,'addr:province'] = address_check[:3]
  #3番目の文字が"府"の場合
  elif address_check[2:3] == "府":
    #3番めまでの文字を'addr:province'に格納する。
    df2.at[index,'addr:province'] = address_check[:3]
  #3番目の文字が"県"の場合
  elif address_check[2:3] == "県":
    #3番めまでの文字を'addr:province'に格納する。
    df2.at[index,'addr:province'] = address_check[:3]
  #4番目の文字が"県"の場合
  elif address_check[3:4] == "県":
    #4番めまでの文字を'addr:province'に格納する。
    df2.at[index,'addr:province'] = address_check[:4]

#出力に必要な列だけ選択する:df3
df3 = df2.loc[:,['lat','lon', 'name', 'ref:gbfs', 'capacity', 'addr:province']]

#df3にOSM用の固定値を追加
df3.loc[:,'amenity'] = 'bicycle_rental'
df3.loc[:,'brand'] = 'HELLO CYCLING'
df3.loc[:,'brand:ja'] = 'ハローサイクリング'
df3.loc[:,'brand:wikidata'] = 'Q91231927'
df3.loc[:,'network'] = 'HELLO CYCLING'
df3.loc[:,'network:ja'] = 'ハローサイクリング'
df3.loc[:,'network:wikidata'] = 'Q91231927'
df3.loc[:,'contact:website'] = 'https://www.hellocycling.jp'
df3.loc[:,'authentication:app'] = 'yes'
df3.loc[:,'fee'] = 'yes'

#ファイル出力
df3.to_csv('/content/drive/MyDrive/python/h_stations.csv')
#確認用のexcelファイル
df3.to_excel('/content/drive/MyDrive/python/h_stations.xlsx', engine='xlsxwriter')
#データ数出力
print(len(df2))


6491


備考
HelloCyclingのデータでstation_informationのすべてのデータを出力したら、
「IllegalCharacterError」のエラーが出た。
その回避のために「df.to_excel("test.xlsx", engine='xlsxwriter')」のようにengineでxlsxwriterを指定するとでなくなる。
なお、xlsxwriterはデフォルトではGoogle Colaboratoryに入ってないので、pipでインストールが必要。
pip install xlsxwriter
